In [27]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [28]:
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [29]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(inital)

def bias_variable(shape):
    inital = tf.constant(0.1, shape=shape)
    return tf.Variable(inital)


def conv2d(x, W):
    # stride[1,x_movement, y_movement, 1]
    #Must have strides[0] = strides[3]=1
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')  

def max_pool_2x2(x):
    # stride[1,x_movement, y_movement, 1]
    #Must have strides[0] = strides[3]=1
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

In [30]:
xs = tf.placeholder(tf.float32, [None, 784]) #28*28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1]) #图片28*28 ，1为channel
#print(x_image.shape)  #[n_samples,28,28,1]

## conv1 layer
W_conv1 = weight_variable([5, 5, 1, 32])  #patch 5*5, in size 1 图像的厚度，因为是黑白的图像，所以厚度为1, out size 32——卷积层的个数
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  #output size 28*28*32 SAME形式，所以长、宽没有变，size高度变成32
h_pool1 = max_pool_2x2(h_conv1)   #output size 14*14*32

##conv2 layer
W_conv2 = weight_variable([5, 5, 32, 64])  #patch 5*5, in size 32, out size 64——卷积层的个数
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  #output size 14*14 SAME形式，所以长、宽没有变，size高度变成32
h_pool2 = max_pool_2x2(h_conv2)   #output size 7*7*64


## func1 layer
W_fc1 = weight_variable([7*7*64, 1024]) 
b_fc1 = bias_variable([1024])
#[n_shamples,7,7,64] >> [n_shamples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64]) 
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) 
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


### func2 layer
W_fc2 = weight_variable([1024, 10]) 
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [31]:
# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images, mnist.test.labels))

0.106
0.8039
0.8906
0.9064
0.9237
0.9335
0.9392
0.9419
0.9465
0.9464
0.9525
0.9501
0.957
0.9588
0.96
0.9612
0.9622
0.9645
0.9661
0.9662
